# Code:
(Seed is the input text. The predicted text is at the end of the output)

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense

# Read dataset
with open('/content/drive/MyDrive/IndiaUS.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Tokenizer process
tokenizer = Tokenizer()
# fit
tokenizer.fit_on_texts([text])
# assign length of word index
total_words = len(tokenizer.word_index) + 1

# check the tokens
tokenizer.word_index

# declare ngrams
input_sequences = []
# split the sentence from '\n'
for line in text.split('\n'):
    # get tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Maximum sentence length
max_sequence_len = max([len(seq) for seq in input_sequences])

# Input sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Determine X and y parameters in
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Convert one-hot-encode
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

# Create model with both GRU and LSTM layers
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(GRU(100, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))  # to convert predicted scores to probabilities
print(model.summary())

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=100, verbose=1)

# Determine a text
seed_text = "The two sides were not expected to"
# Predict word number
next_words = 15

for _ in range(next_words):
    # Convert to token
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    # Pad sequences
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    # Model prediction
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    # Get predicted words
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 82, 100)           22000     
                                                                 
 gru (GRU)                   (None, 82, 100)           60600     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense_2 (Dense)             (None, 220)               33220     
                                                                 
Total params: 266420 (1.02 MB)
Trainable params: 266420 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/100
12/12 [==============================] - 7s 209ms/step - loss: 5.3755 - accuracy: 0.0615
Epoch 2/100
12/12 [=========================

# Model accuracy

In [ ]:
loss, accuracy = model.evaluate(X, y)
print('Test Accuracy: {:.2f}%'.format(accuracy * 100))

12/12 [==============================] - 6s 188ms/step - loss: 0.2220 - accuracy: 0.9947
Test Accuracy: 99.47%
